In [1]:
import importlib

import mysklearn.myutils
importlib.reload(mysklearn.myutils)
import mysklearn.myutils as myutils


import mysklearn.mypytable
importlib.reload(mysklearn.mypytable)
from mysklearn.mypytable import MyPyTable 


import mysklearn.myclassifiers
importlib.reload(mysklearn.myclassifiers)
from mysklearn.myclassifiers import MyKNeighborsClassifier, MyDummyClassifier, MyNaiveBayesClassifier, MyDecisionTreeClassifier

import mysklearn.myevaluation
importlib.reload(mysklearn.myevaluation)
import mysklearn.myevaluation as myevaluation

import copy

In [3]:
df = MyPyTable()
df.load_from_file("high_diamond_ranked_10min.csv")
y = df.get_column("blueWins")
X = df.data.copy()
for entry in X:
    del entry[0]
    del entry[0]
knn_clf = MyKNeighborsClassifier()
nb_clf = MyNaiveBayesClassifier()
dummy_clf = MyDummyClassifier()
tree_clf = MyDecisionTreeClassifier()

myutils.compute_clf_stats(knn_clf,X,y,"kNN Classifier")
myutils.compute_clf_stats(dummy_clf,X,y,"Dummy Classifier")


Stratified 10-Fold Cross Validation
kNN Classifier : accuracy =  0.6743597530114384  error =  0.32564024698856164
binary precision:  0.6755168220510742
binary recall:  0.673469387755102
f1 score:  0.6744915511484367


ValueError: 0.0 is not in list

In [6]:
print(df.column_names)
print("Distribution of blueWins in the dataset:")
print("Blue team wins: ",y.count(1))
print("Blue team loses: ",y.count(0))

['gameId', 'blueWins', 'blueWardsPlaced', 'blueWardsDestroyed', 'blueFirstBlood', 'blueKills', 'blueDeaths', 'blueAssists', 'blueEliteMonsters', 'blueDragons', 'blueHeralds', 'blueTowersDestroyed', 'blueTotalGold', 'blueAvgLevel', 'blueTotalExperience', 'blueTotalMinionsKilled', 'blueTotalJungleMinionsKilled', 'blueGoldDiff', 'blueExperienceDiff', 'blueCSPerMin', 'blueGoldPerMin', 'redWardsPlaced', 'redWardsDestroyed', 'redFirstBlood', 'redKills', 'redDeaths', 'redAssists', 'redEliteMonsters', 'redDragons', 'redHeralds', 'redTowersDestroyed', 'redTotalGold', 'redAvgLevel', 'redTotalExperience', 'redTotalMinionsKilled', 'redTotalJungleMinionsKilled', 'redGoldDiff', 'redExperienceDiff', 'redCSPerMin', 'redGoldPerMin']
Distribution of blueWins in the dataset:
Blue team wins:  4930
Blue team loses:  4949
